In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pytz
import ta

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from datetime import datetime
import MetaTrader5 as mt5

from ortisan_ta.dataaccess import DataItem, MetaTraderDataAccess
import ortisan_ta.utils.analysis as ortisan_ta

from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volume import OnBalanceVolumeIndicator
from ortisan_ta.simulator import MarketSimulator


In [2]:
TOP_50_ASSETS_IBOVESPA = ["ABEV3",
"AZUL4",
"B3SA3",
"BBAS3",
"BBDC4",
"BBSE3",
"BPAC11",
"BRDT3",
"BRFS3",
"BRML3",
"BTOW3",
"CCRO3",
"CIEL3",
"CMIG4",
"COGN3",
"CSNA3",
"CVCB3",
"CYRE3",
"ELET3",
"EQTL3",
"GGBR4",
"GNDI3",
"GOAU4",
"GOLL4",
"IRBR3",
"ITSA4",
"ITUB4",
"JBSS3",
"KLBN11",
"LAME4",
"LREN3",
"MGLU3",
"MRFG3",
"MULT3",
"NTCO3",
"PETR3",
"PETR4",
"PRIO3",
"RADL3",
"RAIL3",
"RENT3",
"SBSP3",
"SULA11",
"SUZB3",
"TOTS3",
"UGPA3",
"USIM5",
"VALE3",
"VVAR3",
"WEGE3"]

### Load data from 1 year

In [3]:
data_access = MetaTraderDataAccess()
dfs = data_access.get_rates_from_symbols(TOP_50_ASSETS_IBOVESPA, datetime(2020, 1, 1), datetime(2021, 1, 1), mt5.TIMEFRAME_D1)
dfs

{'ABEV3':                       Open   High    Low  Close  Spread    Volume
 Date                                                             
 2020-01-01 21:00:00  18.86  19.25  18.78  19.20       1  16411300
 2020-01-02 21:00:00  18.95  19.16  18.89  18.93       1  20059000
 2020-01-05 21:00:00  18.86  19.17  18.69  19.02       1  14644300
 2020-01-06 21:00:00  19.05  19.23  18.86  19.06       1  18036600
 2020-01-07 21:00:00  19.09  19.16  18.86  18.97       1  14051700
 ...                    ...    ...    ...    ...     ...       ...
 2020-11-29 21:00:00  14.42  14.50  13.98  13.98       1  41880800
 2020-11-30 21:00:00  14.09  14.74  14.08  14.68       1  26702100
 2020-12-01 21:00:00  14.61  14.93  14.48  14.88       1  18287100
 2020-12-02 21:00:00  14.74  14.86  14.30  14.70       1  32918200
 2020-12-03 21:00:00  14.74  14.81  14.56  14.74       1  18361200
 
 [233 rows x 6 columns],
 'AZUL4':                       Open   High    Low  Close  Spread    Volume
 Date            

### Join all closes into one dataframe. Each column is the symbol

In [18]:
dict_closes = {symbol: df.Close for (symbol, df) in dfs.items()}
df_closes = pd.DataFrame.from_dict(dict_closes, orient='columns')
df_closes.head()

,ABEV3,AZUL4,B3SA3,BBAS3,BBDC4,BBSE3,BPAC11,BRDT3,BRFS3,BRML3,...,RENT3,SBSP3,SULA11,SUZB3,TOTS3,UGPA3,USIM5,VALE3,VVAR3,WEGE3
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01 21:00:00,19.20,58.80,43.56,52.53,33.30,33.82,74.89,29.57,35.88,18.22,...,49.30,58.87,58.47,40.79,22.95,25.28,9.61,53.07,11.73,34.97
2020-01-02 21:00:00,18.93,56.76,42.30,52.44,33.32,33.44,74.94,29.40,35.94,18.63,...,48.59,57.86,60.84,40.65,23.43,24.98,9.51,52.68,11.48,34.53
2020-01-05 21:00:00,19.02,55.00,41.90,51.74,32.72,33.99,74.06,27.93,35.57,18.50,...,48.02,56.97,60.43,40.58,22.90,24.44,9.33,52.36,11.48,34.62
2020-01-06 21:00:00,19.06,56.82,43.25,51.35,32.15,34.01,74.82,27.89,35.16,18.80,...,48.49,56.00,59.62,41.10,23.00,24.98,9.42,52.74,11.65,34.88
2020-01-07 21:00:00,18.97,56.92,43.14,50.89,31.65,33.99,74.33,27.76,36.51,18.25,...,48.25,55.17,59.41,43.15,22.69,24.79,9.30,52.75,11.60,33.56


### Calculate correlation between all symbols

In [20]:
corr = df_closes.corr()

,ABEV3,AZUL4,B3SA3,BBAS3,BBDC4,BBSE3,BPAC11,BRDT3,BRFS3,BRML3,...,RENT3,SBSP3,SULA11,SUZB3,TOTS3,UGPA3,USIM5,VALE3,VVAR3,WEGE3
ABEV3,1.000000,0.903876,0.118063,0.925415,0.949326,0.913985,0.493867,0.873020,0.878721,0.871432,...,0.402191,0.686094,0.879174,0.054643,0.284292,0.821189,0.436074,0.177076,0.232178,-0.098410
AZUL4,0.903876,1.000000,0.064191,0.946992,0.948759,0.920657,0.428634,0.863733,0.869204,0.908294,...,0.492309,0.558399,0.823925,0.113153,0.321151,0.831846,0.521472,0.212690,0.174233,-0.053239
B3SA3,0.118063,0.064191,1.000000,0.095070,0.135849,0.171587,0.900904,0.237527,-0.055622,-0.162582,...,0.697403,0.479972,0.137733,0.563086,0.852126,0.444698,0.646429,0.753319,0.940506,0.811551
BBAS3,0.925415,0.946992,0.095070,1.000000,0.968725,0.958007,0.448759,0.939519,0.910147,0.952925,...,0.374780,0.724378,0.898116,-0.009226,0.282913,0.814726,0.405253,0.120994,0.190150,-0.143342
BBDC4,0.949326,0.948759,0.135849,0.968725,1.000000,0.964678,0.492621,0.873644,0.879795,0.913514,...,0.470358,0.652615,0.847737,0.044479,0.287485,0.808785,0.460933,0.267343,0.192634,-0.037270
BBSE3,0.913985,0.920657,0.171587,0.958007,0.964678,1.000000,0.519506,0.891118,0.880247,0.901692,...,0.417597,0.714255,0.879781,0.018266,0.309082,0.815295,0.436243,0.222444,0.220819,-0.067859
BPAC11,0.493867,0.428634,0.900904,0.448759,0.492621,0.519506,1.000000,0.545719,0.317060,0.206192,...,0.800763,0.656999,0.449580,0.561264,0.894681,0.742411,0.790534,0.751230,0.920138,0.698843
BRDT3,0.873020,0.863733,0.237527,0.939519,0.873644,0.891118,0.545719,1.000000,0.893111,0.871971,...,0.403230,0.820354,0.918322,0.084173,0.440680,0.886941,0.454808,0.131852,0.366330,-0.064900
BRFS3,0.878721,0.869204,-0.055622,0.910147,0.879795,0.880247,0.317060,0.893111,1.000000,0.911953,...,0.199472,0.657021,0.926838,0.002155,0.153537,0.773153,0.253253,0.001170,0.067225,-0.330745
BRML3,0.871432,0.908294,-0.162582,0.952925,0.913514,0.901692,0.206192,0.871971,0.911953,1.000000,...,0.131277,0.644591,0.884820,-0.209089,0.050584,0.679051,0.174290,-0.192959,-0.051672,-0.421888


In [24]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_lowest_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=True)
    return au_corr[0:n]

print("Lowest Absolute Correlations")
print(get_lowest_abs_correlations(corr, 5))

Lowest Absolute Correlations
ITSA4  PRIO3    0.003119
UGPA3  USIM5    0.003741
BBDC4  PRIO3    0.006079
CSNA3  CYRE3    0.007125
AZUL4  PRIO3    0.007285
dtype: float64
